In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import pandas as pd
import nltk

In [10]:
import scipy

In [2]:
video_data = pd.read_csv("dataset/USvideos.csv")
video_data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [20]:
clean_vid_data = video_data.groupby(["title"]).min().reset_index()
clean_vid_data

,title,video_id,trending_date,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed
0,#184 Making a PCB using EasyEDA. // Review,BPmgDhwbd1w,17.07.12,MickMake,28,2017-12-02T14:05:07.000Z,"MickMake|""electronics""|""embedded""|""maker""|""diy...",3237,161,2,35,https://i.ytimg.com/vi/BPmgDhwbd1w/default.jpg,False,False,False
1,"#23 Feed The Homeless | One List, One Life",4qakFfGRV4E,17.01.12,"One List , One Life",22,2017-11-30T15:36:12.000Z,"homeless|""experiment""|""people""|""man""|""singing""...",32385,568,77,97,https://i.ytimg.com/vi/4qakFfGRV4E/default.jpg,False,False,False
2,#57: AM and SSB explained,l7n58h-Zj3I,18.04.01,Radio Physics and Electronics,28,2017-12-24T19:15:29.000Z,"AM|""SSB""|""amplitude modulation""|""single sideba...",3287,79,3,19,https://i.ytimg.com/vi/l7n58h-Zj3I/default.jpg,False,False,False
3,"#AboveTheNoise feat. Serena Williams, Neymar J...",oWithLP0VlQ,17.29.11,Beats by Dre,10,2017-11-22T17:23:20.000Z,[none],2152261,7824,158,324,https://i.ytimg.com/vi/oWithLP0VlQ/default.jpg,False,False,False
4,#DisneyParksLIVE: Fantasy In The Sky New Years...,FQp6F0PjfpI,18.06.01,Disney Parks,24,2018-01-01T05:11:37.000Z,"Fantasy In The Sky New Years Eve Fireworks|""Di...",130416,1812,30,110,https://i.ytimg.com/vi/FQp6F0PjfpI/default.jpg,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6450,힐링쿠킹♥ 빙글빙글 회오리젤리 만들기! - Ari Kitchen(아리키친),ghNoCu0QiH4,18.04.02,ARIKITCHEN (아리키친),22,2018-02-03T01:09:48.000Z,"아리키친|""arikitchen""|""아리키친 젤리""|""회오리젤리""|""jello rol...",250007,7854,133,2562,https://i.ytimg.com/vi/ghNoCu0QiH4/default.jpg,False,False,False
6451,"🍒 april favs 🍒 (music, clothes, life)",IUsFWiE0d20,18.01.05,Conan Gray,22,2018-04-22T16:00:01.000Z,"conanxcanon|""joeygraceffa""|""connor""|""franta""|""...",128633,17483,42,1867,https://i.ytimg.com/vi/IUsFWiE0d20/default.jpg,False,False,False
6452,🍫달콤 쌉사름~ 리얼 초콜릿 우유 만들기 Real Chocolate Milk | 한세,CNBGEWRdeIM,18.10.02,HANSE 한세,26,2018-02-09T10:36:13.000Z,"CookingASMR|""FoodASMR""|""베이킹""|""ASMR""|""ZOOMH6""|""...",84325,4467,21,576,https://i.ytimg.com/vi/CNBGEWRdeIM/default.jpg,False,False,False
6453,🎃 How to make Pumpkin Pie Mistakes,bAfn2duIlN8,17.22.11,iJustine,22,2017-11-21T19:39:43.000Z,"ijustine|""how to make pumpkin pie""|""pumpkin pi...",76859,4895,164,718,https://i.ytimg.com/vi/bAfn2duIlN8/default.jpg,False,False,False


In [4]:
video = clean_vid_data[["title", "category_id"]]
video["title_clean"] = video.title.str.lower()
video["title_clean"] = video.title_clean.str.replace('[^a-zA-Z ]', '')
video

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,category_id,title_clean
0,#184 Making a PCB using EasyEDA. // Review,28,making a pcb using easyeda review
1,"#23 Feed The Homeless | One List, One Life",22,feed the homeless one list one life
2,#57: AM and SSB explained,28,am and ssb explained
3,"#AboveTheNoise feat. Serena Williams, Neymar J...",10,abovethenoise feat serena williams neymar jr c...
4,#DisneyParksLIVE: Fantasy In The Sky New Years...,24,disneyparkslive fantasy in the sky new years e...
...,...,...,...
6450,힐링쿠킹♥ 빙글빙글 회오리젤리 만들기! - Ari Kitchen(아리키친),22,ari kitchen
6451,"🍒 april favs 🍒 (music, clothes, life)",22,april favs music clothes life
6452,🍫달콤 쌉사름~ 리얼 초콜릿 우유 만들기 Real Chocolate Milk | 한세,26,real chocolate milk
6453,🎃 How to make Pumpkin Pie Mistakes,22,how to make pumpkin pie mistakes


### Three methods to calulate the similarity between two titles
     get_jaccard_sim and get_cosine_sim measure the count of words that appear in two titles while get_gensim_sim calculates the similarity using semantic meaning from a pre-trained model




In [49]:
#source: https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [56]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)[0][1]
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [75]:
#!pip install gensim
import gensim
import gensim.downloader as api
#word_vectors = api.load("glove-wiki-gigaword-100")
def get_gensim_sim(first, second):
    first = first.split()
    second = second.split()
    try:
        similarity = word_vectors.n_similarity(first.split(), second.split())
    except:
        new_first = []
        new_second = []
        
        for f in first:
            if f in word_vectors:
                new_first += [f]
        for s in second:
            if s in word_vectors:
                new_second += [s]
        similarity = word_vectors.n_similarity(new_first or ['none'], new_second or ['none'])
    return similarity


#### Outputs the top three titles in the same category

In [76]:
def find_similar_title(v, function):
    category = video.loc[video.title_clean.str.endswith(v)]["category_id"].values[0]
    video_category = video.loc[video['category_id']==category]

    score = []
    
    for t in video_category.title_clean:
        score += [function(v, t)]
    video_category["score"] = score
    
    video_category = video_category.loc[video_category['score']< 1].sort_values(by=['score'], ascending=False)
    #return score
    return video_category.title[0:3]
find_similar_title("iphone case", get_gensim_sim)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


6384    iPhone X – Selfies on iPhone X – Apple
529            Apple iPhone X - One Week Later
530       Apple iPhone X Review: The Best Yet!
Name: title, dtype: object